# Question & Answer System
by ES

In [100]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import urllib
import sys
import os
import zipfile
import tarfile
import json 
import hashlib
import re
import itertools
import gensim
import string
import math
import nltk
import random
import csv
import io
import spacy
import scipy
from gensim.summarization import bm25
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from importlib import reload
# from spacy.en import English
from collections import Counter
from nltk import pos_tag, word_tokenize
from nltk.tree import Tree
# reload(sys)
# sys.setdefaultencoding('utf-8')

In [105]:
with open('training.json','r') as f:
    training = json.load(f)

with open('documents.json','r') as f:
    documents = json.load(f)

with open('devel.json','r') as f:
    develop = json.load(f)
    
with open('testing.json','r') as f:
    testing = json.load(f)


documents_dict = {}
t = str(len(documents))
num = 0
for doc in documents:
    sys.stdout.write(('getting documents dict: '+'{0}/'+t+'\r').format(num + 1))
    sys.stdout.flush()
    para_list = []
    for para in doc["text"]:
        word_list = word_tokenize(para)
        para_list.append(word_list)
    documents_dict[doc["docid"]]=para_list
    num+=1

In [107]:
develop_data = []
t = str(len(develop))
num = 0
for question in develop:
    sys.stdout.write(('getting develop data: '+'{0}/'+t+'\r').format(num + 1))
    sys.stdout.flush()
    bm25Model = bm25.BM25(documents_dict[question["docid"]])
    average_idf = sum(map(lambda k: float(bm25Model.idf[k]), bm25Model.idf.keys())) / len(bm25Model.idf.keys())
    scores = bm25Model.get_scores(word_tokenize(question["question"]),average_idf)
    idx = scores.index(max(scores))
    
    
    develop_data.append((question["question"],documents[question["docid"]]["text"][idx],idx,question["answer_paragraph"],question["text"]))
    num+=1

In [53]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
model = gensim.models.Word2Vec(sentences, min_count=1)
print(type(model))
# print(model.shape)

<class 'gensim.models.word2vec.Word2Vec'>


In [108]:
def tfidfAcurate(data_list):
    data_count = 0
    correct_count = 0
    for data in data_list:
#         print(data)
        data_count+=1
#         print(data[2],data[3])
        if data[2]==data[3]:
            correct_count +=1
    return float(correct_count)/data_count

print(tfidfAcurate(develop_data))

0.7203745560219568


In [109]:
def getEntity(sentence):
    ## spacy
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(sentence)
    return doc.ents

In [110]:
def getTextByLabel(entity,label_list):
    answer_list = []
    for ent in entity:
        label = ent.label_
        if label in label_list:
            answer_list.append(ent.text)
    return answer_list

def getAnswer(data_list):
    t = str(len(data_list))
    now = 0
    answer=[]
    id_num = 0
    count = [0,0,[0,0,0],[0,0,0,0],0,[0,0,0,0],0]
    useList = [0,0]
    for data in data_list:
        sys.stdout.write(('{0}/'+t+'\r').format(id_num + 1))
        sys.stdout.flush()
        paragraph_entity = getEntity(data[1])
        answer_list = list()
        
        if bool(re.search('who',data[0],re.IGNORECASE)):
            count[0]+=1
            answer_list = getTextByLabel(paragraph_entity,["PERSON","NORP","ORG"])
        elif bool(re.search('where',data[0],re.IGNORECASE)):
            count[1]+=1
            answer_list = getTextByLabel(paragraph_entity,["FACILITY","GPE","LOC"])
        elif bool(re.search('what',data[0],re.IGNORECASE)):
            if bool(re.search('what time',data[0],re.IGNORECASE)):
                count[2][0]+=1
                answer_list = getTextByLabel(paragraph_entity,["DATE","TIME"])
            elif bool(re.search('what year',data[0],re.IGNORECASE) or re.search('what day',data[0],re.IGNORECASE) or re.search('what date',data[0],re.IGNORECASE)):
                count[2][1]+=1
                answer_list = getTextByLabel(paragraph_entity,["DATE"])
            elif bool(re.search('what city',data[0],re.IGNORECASE or re.search('what country',data[0],re.IGNORECASE)):
                count[2][2]+=1
                answer_list = getTextByLabel(paragraph_entity,["GPE"])
            else:
                count[2][2]+=1
                answer_list = getTextByLabel(paragraph_entity,["FACILITY","PRODUCT","WORK_OF_ART","EVENT","ORG"])
        elif bool(re.search('how',data[0],re.IGNORECASE)):
            if bool(re.search('how much',data[0],re.IGNORECASE)):
                count[3][0]+=1
                answer_list = getTextByLabel(paragraph_entity,["MONEY"])
            if bool(re.search('how many',data[0],re.IGNORECASE)):
                count[3][1]+=1
                answer_list = getTextByLabel(paragraph_entity,["MONEY","QUANTITY","CARDINAL"])
            if bool(re.search('how does',data[0],re.IGNORECASE) or re.search('how was',data[0],re.IGNORECASE)):
                count[3][2]+=1
                answer_list = getTextByLabel(paragraph_entity,["LAW","EVENT","PRODUCT","PERSON"])
            else:
                count[3][3]+=1
                answer_list = getTextByLabel(paragraph_entity,["MONEY","QUANTITY","CARDINAL"])
        elif bool(re.search('when',data[0],re.IGNORECASE)):
            count[4]+=1
            answer_list = getTextByLabel(paragraph_entity,["DATE","TIME"])
        elif bool(re.search('which',data[0],re.IGNORECASE)):
            if bool(re.search('which person',data[0],re.IGNORECASE)):
                count[5][0]+=1
                answer_list = getTextByLabel(paragraph_entity,["PERSON"])
            elif bool(re.search('which time',data[0],re.IGNORECASE)):
                count[5][1]+=1
                answer_list = getTextByLabel(paragraph_entity,["DATE","TIME"])
            elif bool(re.search('which country',data[0],re.IGNORECASE)):
                count[5][2]+=1
                answer_list = getTextByLabel(paragraph_entity,["NORP","ORG"])
            else:
                count[5][3]+=1
                answer_list = getTextByLabel(paragraph_entity,["FACILITY","PRODUCT","WORK_OF_ART","EVENT","ORG"])
        else:
            count[6]+=1
        if len(answer_list)==0:
            useList[1]+=1
            try:
                index = random.randint(0,len(paragraph_entity)-1) 
                answer.append (([str(id_num)+','+paragraph_entity[index].text.replace(",", "")],data[2]))
            except:
                index = random.randint(0,len(data[1])-1)
                answer.append(([str(id_num)+','+data[1][index].replace(",", "")],data[2]))
        else:
            useList[0]+=1
            index = random.randint(0,len(answer_list)-1)
            answer.append (([str(id_num)+','+answer_list[index].replace(",", "")],data[2]))
        id_num +=1
    print("count ",count)
    print("use list ",useList)
    return answer

    

# for line in answer:
#     print(line)
# count  [2198, 279, 422, 156, 563]
# use list  [2757, 861]

In [ ]:
# answer = getAnswer(test_data)
answer = getAnswer(test_data)

In [11]:
def campare(question,answer):
    if question[4]==str(answer[0]).strip("'").split(',')[1][:-2]:
        return True
    else:
        return False

def answerAcurate(data_list,answer):
    data_count = 0
    correct_count = 0
    results = map(campare,data_list,answer)
    for result in results:
        data_count+=1
        if result:
            correct_count +=1
    return float(correct_count)/data_count

print(answerAcurate(develop_data,answer))
    

In [25]:
##Write into csv file
def writeFile(answer):
#     sys.stdout = io.TextIOWrapper(sys.stdout.buffer,encoding='utf8')
    out = open('answer.csv','a',newline='')
    csv_write =csv.writer(out,dialect='excel')
    csv_write.writerow(["id,answer"])
    answerID = 0
    for line in answer:
        try:
            csv_write.writerow(line[0])
        except:
            print("line "+str(answerID)+" cannot write： "+str(line[0]))
        answerID+=1
    out.close()
    
writeFile(answer)


line 81 cannot write： ['81,Galschiøt']
line 136 cannot write： ['136,Galschiøt']
line 220 cannot write： ['220,Galschiøt']
line 222 cannot write： ['222,Galschiøt']
line 261 cannot write： ['261,Điếu Cày']
line 558 cannot write： ['558,£100m']
line 806 cannot write： ['806,Aškuzai/Iškuzai']
line 835 cannot write： ['835,Ποσειδώνας']
line 1075 cannot write： ['1075,Calçada Portuguesa']
line 1095 cannot write： ['1095,Calçada Portuguesa']
line 1096 cannot write： ['1096,the River Mureş']
line 1163 cannot write： ['1163,Calçada Portuguesa']
line 1166 cannot write： ['1166,Calçada Portuguesa']
line 1353 cannot write： ['1353,Åland Islands']
line 1928 cannot write： ['1928,Mötley Crüe']
line 2188 cannot write： ["2188,François Bernier's"]
line 2259 cannot write： ['2259,Štrkalj']
line 2328 cannot write： ["2328,François Bernier's"]
line 2388 cannot write： ['2388,the Norte Chico Civilization Formative Mesoamerica and Ancient Hawaiʻi']
line 2392 cannot write： ['2392,the Norte Chico Civilization Formative Meso